In [137]:
import pandas as pd
import numpy
from nltk.corpus import stopwords
import textblob 
import enchant
import nltk
stemmer = nltk.stem.SnowballStemmer('english')

In [140]:
def get_data(filename):
    df= pd.read_csv(filename)
    df = df.set_index('essay_id')
    df = df[['essay','rater1_domain1']]
    stop = stopwords.words('english')
    df['essay'] = df['essay'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df['essay'] = df['essay'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split() if '@' not in word]))
    return df

In [141]:
essay_set_1 = get_data('essay_set_1.csv')
essay_set_2 = get_data('essay_set_2.csv')
essay_set_7 = get_data('essay_set_7.csv')
essay_set_8 = get_data('essay_set_8.csv')
essay_set_1.head()

,essay,rater1_domain1
essay_id,,
1,"dear local newspaper, i think effect comput pe...",4.0
2,dear i believ use comput benefit us mani way l...,5.0
3,"dear, more peopl use computers, everyon agre b...",4.0
4,"dear local newspaper, i found mani expert say ...",5.0
5,dear i know comput posit effect people. the co...,4.0


In [142]:
essay_set_1['vocab'] = 0
essay_set_2['vocab'] = 0
essay_set_7['vocab'] = 0
essay_set_8['vocab'] = 0

essay_set_1.loc[essay_set_1.rater1_domain1 >= 3, 'vocab'] = 1
essay_set_1.loc[essay_set_1.rater1_domain1 >= 5, 'vocab'] = 2
essay_set_1.drop('rater1_domain1',inplace=True,axis=1)

essay_set_2.loc[essay_set_2.rater1_domain1 >= 3, 'vocab'] = 1
essay_set_2.loc[essay_set_2.rater1_domain1 >= 5, 'vocab'] = 2
essay_set_2.drop('rater1_domain1',inplace=True,axis=1)

essay_set_7.loc[essay_set_7.rater1_domain1 >= 5, 'vocab'] = 1
essay_set_7.loc[essay_set_7.rater1_domain1 >= 10, 'vocab'] = 2
essay_set_7.drop('rater1_domain1',inplace=True,axis=1)

essay_set_8.loc[essay_set_8.rater1_domain1 >= 11, 'vocab'] = 1
essay_set_8.loc[essay_set_8.rater1_domain1 >= 21, 'vocab'] = 2
essay_set_8.drop('rater1_domain1',inplace=True,axis=1)

In [143]:
essay_set_1.describe()

,vocab
count,1783.000000
mean,1.330342
std,0.513799
min,0.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,2.000000


In [144]:
frames = [essay_set_1, essay_set_2, essay_set_7, essay_set_8]
df = pd.concat(frames)
df = df.reindex(numpy.random.permutation(df.index))
df.head()

,essay,vocab
essay_id,,
4145,"freedom chose someth everi person have, type c...",1
136,"there mani differ side comput problem, well it...",1
3491,i person dont think censorship book come i thi...,0
3612,"what library? a librari place go books, movies...",1
3722,"books, music, movies, magazines, we'v pleasur ...",2


In [145]:
import numpy
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(df['essay'].values)

In [146]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('classifier',  MultinomialNB()) ])

pipeline.fit(df['essay'].values, df['vocab'].values)

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [149]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
train,test,tr_y,te_y = train_test_split(df['essay'],df['vocab'], test_size = 0.5)
pipeline.fit(train, tr_y)
predictions = pipeline.predict(test)
vals = [0,1,2]
y_actu = pd.Categorical(te_y, categories=vals)
y_actu = pd.Series(y_actu,name="Actual")
y_pred = pd.Categorical(predictions, categories=vals)
y_pred = pd.Series(y_pred,name="Prediction")
print('Accuracy Score:', accuracy_score(y_actu, y_pred))
pd.crosstab(y_actu, y_pred,margins = True)

Accuracy Score: 0.664397549353


Prediction,0,1,2,All
Actual,,,,
0,3,182,11,196
1,0,1575,540,2115
2,0,253,374,627
All,3,2010,925,2938
